# Data Preprocessing

In [1]:
import numpy as np
import pandas as pd

import math

In [2]:
occupancy_df = pd.read_csv('../TxOccupation_QDS_2015et16.csv', sep=';', decimal=",")

In [3]:
occupancy_df.head()

,NoTroncon,DescriptionTroncon,DateCreation,NoHeure,NbMinDispo,TxOccupation
0,1,Bleury entre Sherbrooke et Président-Kennedy -...,2015-01-01,9,0,0.0
1,1,Bleury entre Sherbrooke et Président-Kennedy -...,2015-01-01,10,0,0.0
2,1,Bleury entre Sherbrooke et Président-Kennedy -...,2015-01-01,11,0,0.0
3,1,Bleury entre Sherbrooke et Président-Kennedy -...,2015-01-01,12,0,0.0
4,1,Bleury entre Sherbrooke et Président-Kennedy -...,2015-01-01,13,0,0.0


# Remove rows with 0 available minutes

In [4]:
occupancy_df = occupancy_df[occupancy_df['NbMinDispo'] > 0]

# Remove columns, change data types

In [5]:
# remove some columns
occupancy_df.drop('DescriptionTroncon', 1, inplace=True)
occupancy_df.drop('NbMinDispo', 1, inplace=True)

# convert NoHeure and NoTroncon columns from int to category
occupancy_df.NoHeure = occupancy_df.NoHeure.astype('int')
occupancy_df.NoTroncon = occupancy_df.NoTroncon.astype('int')

# convert DateCreation from Object to Date
occupancy_df.DateCreation = pd.to_datetime(occupancy_df['DateCreation'])

# add columns day of week and days of year
occupancy_df.insert(loc=2, column='DayOfYear', value=occupancy_df['DateCreation'].dt.dayofyear)
occupancy_df.insert(loc=3, column='DayOfWeek', value=occupancy_df['DateCreation'].dt.dayofweek)

In [6]:
occupancy_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1164294 entries, 12 to 1186355
Data columns (total 6 columns):
NoTroncon       1164294 non-null int64
DateCreation    1164294 non-null datetime64[ns]
DayOfYear       1164294 non-null int64
DayOfWeek       1164294 non-null int64
NoHeure         1164294 non-null int64
TxOccupation    1164294 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 62.2 MB


In [7]:
occupancy_df.head()

,NoTroncon,DateCreation,DayOfYear,DayOfWeek,NoHeure,TxOccupation
12,1,2015-01-02,2,4,9,0.0000
13,1,2015-01-02,2,4,10,0.0000
14,1,2015-01-02,2,4,11,0.0000
15,1,2015-01-02,2,4,12,0.0510
16,1,2015-01-02,2,4,13,0.0588


---

# Create a section-parking spot-gps data frame
 - troncon longitude and lattitude
 - number of parking slot per troncon

In [8]:
emplacements = pd.read_csv('../emplacement_gps.csv', sep=',', decimal=".")

In [9]:
emplacements.head()

,sNoEmplacement,nNoTroncon,longitude,latitude
0,G406,1,-73.571028,45.508322
1,G408,1,-73.570925,45.508273
2,G409,1,-73.570889,45.508256
3,G411,1,-73.570721,45.508175
4,G412,1,-73.570619,45.508130


In [10]:
emplacements.rename(columns = {'nNoTroncon':'NoTroncon'}, inplace = True)

In [11]:
troncons_df  = pd.DataFrame(columns = ['NoTroncon', 'CenterLongitude', 'CenterLatitude', 'EmplacementCount'])

for i, no_troncon in enumerate(emplacements.NoTroncon.unique()):
    center_longitude = np.mean(emplacements[emplacements['NoTroncon'] == no_troncon]['longitude'])
    center_latitude = np.mean(emplacements[emplacements['NoTroncon'] == no_troncon]['latitude'])
    emplacement_count = len(emplacements[emplacements['NoTroncon'] == no_troncon])
    
    troncons_df.loc[i] = [
        str(no_troncon), 
        float("{0:.6f}".format(center_longitude)), 
        float("{0:.6f}".format(center_latitude)),
        emplacement_count]
    
troncons_df.NoTroncon = troncons_df.NoTroncon.astype('int')
troncons_df.EmplacementCount = troncons_df.EmplacementCount.astype('int')

In [12]:
troncons_df.head()

,NoTroncon,CenterLongitude,CenterLatitude,EmplacementCount
0,1,-73.570339,45.507996,17
1,2,-73.570659,45.507961,10
2,3,-73.569128,45.507425,6
3,4,-73.569857,45.507581,5
4,5,-73.568403,45.507083,8


In [13]:
troncons_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 0 to 158
Data columns (total 4 columns):
NoTroncon           159 non-null int64
CenterLongitude     159 non-null float64
CenterLatitude      159 non-null float64
EmplacementCount    159 non-null int64
dtypes: float64(2), int64(2)
memory usage: 6.2 KB


---

In [14]:
section_emplacement_gps = pd.merge(troncons_df, emplacements, on='NoTroncon')

In [15]:
section_emplacement_gps.head(18)

,NoTroncon,CenterLongitude,CenterLatitude,EmplacementCount,sNoEmplacement,longitude,latitude
0,1,-73.570339,45.507996,17,G406,-73.571028,45.508322
1,1,-73.570339,45.507996,17,G408,-73.570925,45.508273
2,1,-73.570339,45.507996,17,G409,-73.570889,45.508256
3,1,-73.570339,45.507996,17,G411,-73.570721,45.508175
4,1,-73.570339,45.507996,17,G412,-73.570619,45.508130
5,1,-73.570339,45.507996,17,G413,-73.570619,45.508130
6,1,-73.570339,45.507996,17,G414,-73.570491,45.508069
7,1,-73.570339,45.507996,17,G415,-73.570491,45.508069
8,1,-73.570339,45.507996,17,G416,-73.570361,45.508007
9,1,-73.570339,45.507996,17,G417,-73.570361,45.508007


In [16]:
section_emplacement_gps.to_csv('../section_emplacement_gps.csv', sep=';', index=False)

---

In [17]:
# print (longitude, latitude) for a specific section id
section = section_emplacement_gps[section_emplacement_gps['NoTroncon'] == 1]
section[['longitude', 'latitude']]

,longitude,latitude
0,-73.571028,45.508322
1,-73.570925,45.508273
2,-73.570889,45.508256
3,-73.570721,45.508175
4,-73.570619,45.508130
5,-73.570619,45.508130
6,-73.570491,45.508069
7,-73.570491,45.508069
8,-73.570361,45.508007
9,-73.570361,45.508007


---

# Create Training Set by merging the two data frames

In [18]:
dataset_df = pd.merge(occupancy_df, troncons_df, on='NoTroncon')

In [19]:
dataset_df.head()

,NoTroncon,DateCreation,DayOfYear,DayOfWeek,NoHeure,TxOccupation,CenterLongitude,CenterLatitude,EmplacementCount
0,1,2015-01-02,2,4,9,0.0000,-73.570339,45.507996,17
1,1,2015-01-02,2,4,10,0.0000,-73.570339,45.507996,17
2,1,2015-01-02,2,4,11,0.0000,-73.570339,45.507996,17
3,1,2015-01-02,2,4,12,0.0510,-73.570339,45.507996,17
4,1,2015-01-02,2,4,13,0.0588,-73.570339,45.507996,17


In [20]:
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1153852 entries, 0 to 1153851
Data columns (total 9 columns):
NoTroncon           1153852 non-null int64
DateCreation        1153852 non-null datetime64[ns]
DayOfYear           1153852 non-null int64
DayOfWeek           1153852 non-null int64
NoHeure             1153852 non-null int64
TxOccupation        1153852 non-null float64
CenterLongitude     1153852 non-null float64
CenterLatitude      1153852 non-null float64
EmplacementCount    1153852 non-null int64
dtypes: datetime64[ns](1), float64(3), int64(5)
memory usage: 88.0 MB


# Save basic training set on disk

In [21]:
dataset_df.to_csv('training_set.csv', sep=';', index=False)

---

# Add weather info

In [21]:
weather_df = pd.read_csv('../meteo_montreal.csv', sep=';', decimal=".")

In [22]:
weather_df.head()

,Date,Heure,Temp_mercure,Temp_eolien,Temp,Precip_total_day_mm
0,2015-01-01,0,-6.6,-11.0,-11.0,1.8
1,2015-01-01,1,-7.0,-12.0,-12.0,1.8
2,2015-01-01,2,-7.0,-11.0,-11.0,1.8
3,2015-01-01,3,-7.2,-12.0,-12.0,1.8
4,2015-01-01,4,-7.5,-13.0,-13.0,1.8


In [23]:
weather_df.drop('Temp_mercure', 1, inplace=True)
weather_df.drop('Temp_eolien', 1, inplace=True)

weather_df.rename(columns = {'Heure':'NoHeure'}, inplace = True)

# convert DateCreation from Object to Date
weather_df.rename(columns = {'Date':'DateCreation'}, inplace = True)
weather_df.DateCreation = pd.to_datetime(weather_df['DateCreation'])

In [24]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32136 entries, 0 to 32135
Data columns (total 4 columns):
DateCreation           32136 non-null datetime64[ns]
NoHeure                32136 non-null int64
Temp                   32136 non-null float64
Precip_total_day_mm    32136 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 1004.3 KB


In [25]:
parking_weather_df = pd.merge(dataset_df, weather_df, how='left', left_on=['DateCreation', 'NoHeure'], right_on=['DateCreation', 'NoHeure'])

In [26]:
parking_weather_df.head(24)

,NoTroncon,DateCreation,DayOfYear,DayOfWeek,NoHeure,TxOccupation,CenterLongitude,CenterLatitude,EmplacementCount,Temp,Precip_total_day_mm
0,1,2015-01-02,2,4,9,0.0000,-73.570339,45.507996,17,-11.0,0.1
1,1,2015-01-02,2,4,10,0.0000,-73.570339,45.507996,17,-11.0,0.1
2,1,2015-01-02,2,4,11,0.0000,-73.570339,45.507996,17,-11.0,0.1
3,1,2015-01-02,2,4,12,0.0510,-73.570339,45.507996,17,-10.0,0.1
4,1,2015-01-02,2,4,13,0.0588,-73.570339,45.507996,17,-12.0,0.1
5,1,2015-01-02,2,4,14,0.1029,-73.570339,45.507996,17,-11.0,0.1
6,1,2015-01-02,2,4,15,0.0569,-73.570339,45.507996,17,-10.0,0.1
7,1,2015-01-02,2,4,16,0.0392,-73.570339,45.507996,17,-12.0,0.1
8,1,2015-01-02,2,4,17,0.0000,-73.570339,45.507996,17,-11.0,0.1
9,1,2015-01-02,2,4,18,0.1294,-73.570339,45.507996,17,-11.0,0.1


In [28]:
# show troncon 1
section_1 = parking_weather_df[parking_weather_df['NoTroncon'] == 1]

In [29]:
section_1[section_1['DateCreation'] == "2015-01-02"]

,NoTroncon,DateCreation,DayOfYear,DayOfWeek,NoHeure,TxOccupation,CenterLongitude,CenterLatitude,EmplacementCount,Temp,Precip_total_day_mm
0,1,2015-01-02,2,4,9,0.0000,-73.570339,45.507996,17,-11.0,0.1
1,1,2015-01-02,2,4,10,0.0000,-73.570339,45.507996,17,-11.0,0.1
2,1,2015-01-02,2,4,11,0.0000,-73.570339,45.507996,17,-11.0,0.1
3,1,2015-01-02,2,4,12,0.0510,-73.570339,45.507996,17,-10.0,0.1
4,1,2015-01-02,2,4,13,0.0588,-73.570339,45.507996,17,-12.0,0.1
5,1,2015-01-02,2,4,14,0.1029,-73.570339,45.507996,17,-11.0,0.1
6,1,2015-01-02,2,4,15,0.0569,-73.570339,45.507996,17,-10.0,0.1
7,1,2015-01-02,2,4,16,0.0392,-73.570339,45.507996,17,-12.0,0.1
8,1,2015-01-02,2,4,17,0.0000,-73.570339,45.507996,17,-11.0,0.1
9,1,2015-01-02,2,4,18,0.1294,-73.570339,45.507996,17,-11.0,0.1


In [30]:
section_1[section_1['DateCreation'] == "2016-11-25"]

,NoTroncon,DateCreation,DayOfYear,DayOfWeek,NoHeure,TxOccupation,CenterLongitude,CenterLatitude,EmplacementCount,Temp,Precip_total_day_mm
7326,1,2016-11-25,330,4,9,0.3500,-73.570339,45.507996,17,0.2,2.8
7327,1,2016-11-25,330,4,10,0.4941,-73.570339,45.507996,17,0.3,2.8
7328,1,2016-11-25,330,4,11,0.6235,-73.570339,45.507996,17,0.4,2.8
7329,1,2016-11-25,330,4,12,0.6559,-73.570339,45.507996,17,0.4,2.8
7330,1,2016-11-25,330,4,13,0.6343,-73.570339,45.507996,17,0.6,2.8
7331,1,2016-11-25,330,4,14,0.4598,-73.570339,45.507996,17,0.6,2.8
7332,1,2016-11-25,330,4,15,0.4441,-73.570339,45.507996,17,0.7,2.8
7333,1,2016-11-25,330,4,16,0.5314,-73.570339,45.507996,17,0.8,2.8
7334,1,2016-11-25,330,4,17,0.2853,-73.570339,45.507996,17,0.9,2.8
7335,1,2016-11-25,330,4,18,0.2667,-73.570339,45.507996,17,1.0,2.8


# Save parking and weather training set on disk

In [31]:
parking_weather_df.to_csv('training_set_8_inputs.csv', sep=';', index=False)

---

# Convert TxOccupation into 5 categories

In [32]:
df = pd.read_csv('training_set_8_inputs.csv', sep=';')

In [33]:
df[df['DateCreation'] == "2016-11-25"][:12]

,NoTroncon,DateCreation,DayOfYear,DayOfWeek,NoHeure,TxOccupation,CenterLongitude,CenterLatitude,EmplacementCount,Temp,Precip_total_day_mm
7326,1,2016-11-25,330,4,9,0.3500,-73.570339,45.507996,17,0.2,2.8
7327,1,2016-11-25,330,4,10,0.4941,-73.570339,45.507996,17,0.3,2.8
7328,1,2016-11-25,330,4,11,0.6235,-73.570339,45.507996,17,0.4,2.8
7329,1,2016-11-25,330,4,12,0.6559,-73.570339,45.507996,17,0.4,2.8
7330,1,2016-11-25,330,4,13,0.6343,-73.570339,45.507996,17,0.6,2.8
7331,1,2016-11-25,330,4,14,0.4598,-73.570339,45.507996,17,0.6,2.8
7332,1,2016-11-25,330,4,15,0.4441,-73.570339,45.507996,17,0.7,2.8
7333,1,2016-11-25,330,4,16,0.5314,-73.570339,45.507996,17,0.8,2.8
7334,1,2016-11-25,330,4,17,0.2853,-73.570339,45.507996,17,0.9,2.8
7335,1,2016-11-25,330,4,18,0.2667,-73.570339,45.507996,17,1.0,2.8


In [34]:
df.loc[(df.TxOccupation >= 0.80) & (df.TxOccupation <= 1.00),'TxOccupation'] = 4
df.loc[(df.TxOccupation >= 0.60) & (df.TxOccupation < 0.80), 'TxOccupation'] = 3
df.loc[(df.TxOccupation >= 0.40) & (df.TxOccupation < 0.60), 'TxOccupation'] = 2
df.loc[(df.TxOccupation >= 0.20) & (df.TxOccupation < 0.40), 'TxOccupation'] = 1
df.loc[(df.TxOccupation >= 0.00) & (df.TxOccupation < 0.20), 'TxOccupation'] = 0
df.TxOccupation = df.TxOccupation.astype('int')

In [35]:
df[df['DateCreation'] == "2016-11-25"][:12]

,NoTroncon,DateCreation,DayOfYear,DayOfWeek,NoHeure,TxOccupation,CenterLongitude,CenterLatitude,EmplacementCount,Temp,Precip_total_day_mm
7326,1,2016-11-25,330,4,9,1,-73.570339,45.507996,17,0.2,2.8
7327,1,2016-11-25,330,4,10,2,-73.570339,45.507996,17,0.3,2.8
7328,1,2016-11-25,330,4,11,3,-73.570339,45.507996,17,0.4,2.8
7329,1,2016-11-25,330,4,12,3,-73.570339,45.507996,17,0.4,2.8
7330,1,2016-11-25,330,4,13,3,-73.570339,45.507996,17,0.6,2.8
7331,1,2016-11-25,330,4,14,2,-73.570339,45.507996,17,0.6,2.8
7332,1,2016-11-25,330,4,15,2,-73.570339,45.507996,17,0.7,2.8
7333,1,2016-11-25,330,4,16,2,-73.570339,45.507996,17,0.8,2.8
7334,1,2016-11-25,330,4,17,1,-73.570339,45.507996,17,0.9,2.8
7335,1,2016-11-25,330,4,18,1,-73.570339,45.507996,17,1.0,2.8


In [40]:
from keras.utils import to_categorical

In [37]:
y = df.TxOccupation.values
y_encoded = to_categorical(y)

In [38]:
print(y[:12])
print(y_encoded[:12])

[0 0 0 0 0 0 0 0 0 0 1 0]
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]


---

In [39]:
day = df[df['DateCreation'] == "2016-11-25"]

In [44]:
hour = day[day['NoHeure'] == 9]
print(len(hour))
hour.head()

156


,NoTroncon,DateCreation,DayOfYear,DayOfWeek,NoHeure,TxOccupation,CenterLongitude,CenterLatitude,EmplacementCount,Temp,Precip_total_day_mm
7326,1,2016-11-25,330,4,9,1,-73.570339,45.507996,17,0.2,2.8
15043,2,2016-11-25,330,4,9,0,-73.570659,45.507961,10,0.2,2.8
22760,3,2016-11-25,330,4,9,0,-73.569128,45.507425,6,0.2,2.8
30477,4,2016-11-25,330,4,9,0,-73.569857,45.507581,5,0.2,2.8
38194,5,2016-11-25,330,4,9,0,-73.568403,45.507083,8,0.2,2.8


In [45]:
# loop over rows
for index, row in hour.iterrows():
    print(row.NoTroncon)

1
2
3
4
5
6
7
8
10
12
19
20
21
25
26
27
28
29
30
33
155
156
182
204
205
358
359
360
361
362
363
364
365
366
367
402
403
404
406
407
408
409
410
411
414
415
561
562
563
564
574
585
598
599
612
613
614
615
616
617
618
619
620
621
890
891
892
893
894
895
896
897
898
899
901
902
903
904
905
907
908
909
910
912
913
914
915
916
1378
1379
1417
1419
1420
1436
1437
1438
1439
1457
1458
1461
1462
1463
1464
1465
1466
1512
1529
1567
1568
1592
1593
1595
1596
1622
1623
1624
1625
1686
2020
2028
2031
2034
2038
2039
2276
2284
2375
2578
2592
2593
2601
2612
2613
2618
2619
2626
2627
2628
2716
2717
2718
2719
2720
2721
2732
2770
2771
2772
2773
2774
2775
2781
2821
2874
2918
2919
